# Natural Language Processing (RNN)

In [16]:
import keras
import numpy as np
import pandas as pd
import re

data = pd.read_csv('/Users/yiwenshen/Downloads/songdata.csv')

In [17]:
data.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [14]:
ed = data[data.artist == 'Ed Sheeran']

In [15]:
ed.shape

(53, 4)

In [16]:
text = ed['text'].str.cat(sep=' ').lower()
text = text.replace('\n','')
text = text.replace('\\', '')

In [18]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 29753
Unique characters: 49
Vectorization...


In [19]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [20]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [21]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [24]:
import random
import sys

for epoch in range(1, 30):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(200):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
29753/29753 [==============================] - 51s 2ms/step - loss: 1.2206
--- Generating with seed: "tell me your secrets  give me a friend  let all the good tim"
------ temperature: 0.2
tell me your secrets  give me a friend  let all the good time my my, my, my, my, my, my, my, my, my, my, my, my, my, my, oh give me lover  my, my, my, my, my, my, oh give me lover lover lover lover lover lover lover lover lover lover lover lover lover lover lo
------ temperature: 0.5
ver lover lover lover lover lover lover lover lover lover lover lover lover lover my, my, my, my, my, m-my my, m-my my, gim my lover  and it's not a homeless life for my love like you well me you want me now    then you should learn the fall  feeling on the wa
------ temperature: 1.0
 me now    then you should learn the fall  feeling on the way the flyarme  thongisppeorne is  then're jumt wash't ink dot't  back is and sme bay when you singpt be my sad  and i tell    all the don'r learn my free  . dow ree

he only one that knows i lied  you and i ended over u-n-i  and i said too see the mountain do never let my hutd  and i'm stiltele this in the rock the ones i've been of the road that i'm still way tell me if i'm wresting on the start of the road again  and i'm
------ temperature: 0.5
l me if i'm wresting on the start of the road again  and i'm stiltele the way so love  my, my, my, my, my, my, my, oh give me love, love me love me love, love me love me go mn give me love, love me love my, my, my, my, my, my, my, oh give me love me, my, my, m
------ temperature: 1.0
ve my, my, my, my, my, my, my, oh give me love me, my, my, my, m-my my, m-my my, m-my my, m-my my, m-my my, m-my my, m-my my, m-my my, m-my,my my, my, my, my, mov i jose me sad, use of the road wasting comple never knowshat me  and what i step i'll be druntcro
------ temperature: 1.2
g comple never knowshat me  and what i step i'll be druntcrou chars feettlefred the hume  oh the it dain, ifay hold me, wet bemth  giting ring ba

p off of the ground  and never ever let you down    my eyes    and if the things i'm stull the spenking her and this woo long out,    and i said to make that makes with mountay  and the things i've do now  the motings a chest to you should go nina  then i'm st
------ temperature: 0.5
 now  the motings a chest to you should go nina  then i'm stull if you fall    and i know then again  your eyes  thoughts from the ground  on my thoughts down  that i'm stilt suring coul the fliend  this is not hay.    i don't feelwarnend with your heart out o
------ temperature: 1.0
his is not hay.    i don't feelwarnend with your heart out ot shoolfllong wrine everymemmm through my freends and close your my home me wimt,  my eyes    she keep  and if you filds every save even this senstaits to make up  a live for that i'm stull smart of i
------ temperature: 1.2
s senstaits to make up  a live for that i'm stull smart of it  no my heart out tadle  of all by my cheaatien  of the the as as your hope who thou

/Users/yiwenshen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


the me then oho cheashor tryermire, memoue  what we putey  and ol, yo
epoch 26
Epoch 1/1
29753/29753 [==============================] - 49s 2ms/step - loss: 0.6466: 2s - l
--- Generating with seed: "been what it's about  but maybe we'll go together and just f"
------ temperature: 0.2
been what it's about  but maybe we'll go together and just fire in a the the the start of so on the pain with to stay to say that i'm gone in a cold  and it was not close before in a when i see be wirh i will in the fait to stop  then i'm gone the pocket if i 
------ temperature: 0.5
h i will in the fait to stop  then i'm gone the pocket if i mist you     come and my save if it out  still a sunted my soul the restid from the pony  you'll be the got and we're speaped to say you should got and need to keep mean  but i can't here bustiend sin
------ temperature: 1.0
uld got and need to keep mean  but i can't here bustiend sing,  the way say the  i sunband abyer the the drwsen thing the rights like falling in 